In [1]:
import json


def save(device_map, name):
    with open('model/device_map_%s.json' % name, 'w') as f:
        json.dump(device_map, f)


save(
    {
        'encoder.0': 0,
        'encoder.1': 0,
        'encoder.2': 0,
        'encoder.3': 0,
        'encoder.4': 0,
        'encoder.5.0': 0,
        'encoder.5.1': 'disk',
        'encoder.5.2': 0,
        'encoder.6': 0,
        'encoder.7': 0,
        'encoder.8': 0,
        'encoder.9': 0,
        'decoder.0': 0,
        'decoder.1': 0,
        'decoder.2.0': 0,
        'decoder.2.1': 'disk',
        'decoder.2.2': 0,
        'decoder.3': 0,
        'decoder.4': 0,
        'decoder.5': 0,
        'decoder.6': 0,
        'decoder.7': 0,
        'decoder.8': 0,
        'decoder.9': 0,
    }, 'vae')

In [2]:
def devicemap_Transformer2D(prefix):
    return {
        prefix + 'norm1': 0,
        prefix + 'norm2': 0,
        prefix + 'norm3': 0,
        prefix + 'norm4': 0,
        prefix + 'fc_in': 0,
        prefix + 'fc_out': 0,
        prefix + 'atten1': 'disk',
        prefix + 'atten2': 'disk',
        prefix + 'ff': 0,
    }


def devicemap_TransformerDown(prefix):
    return {
        prefix + 'resnet1': 0,
        prefix + 'resnet2': 0,
        prefix + 'downsampler': 0,
        **devicemap_Transformer2D(prefix + 'transformer1.'),
        **devicemap_Transformer2D(prefix + 'transformer2.'),
    }


def devicemap_TransformerUp(prefix):
    return {
        **devicemap_Transformer2D(prefix + 'transformer.0.'),
        **devicemap_Transformer2D(prefix + 'transformer.1.'),
        **devicemap_Transformer2D(prefix + 'transformer.2.'),
        prefix + 'resnet': 0,
        prefix + 'upsampler': 0,
    }


device_map = {
    'timestep_embedding': 0,
    **devicemap_TransformerDown('down.0.'),
    **devicemap_TransformerDown('down.1.'),
    **devicemap_TransformerDown('down.2.'),
    'down.3': 0,
    **devicemap_Transformer2D('mid.transformer.'),
    'mid.resnet1': 0,
    'mid.resnet2': 0,
    'up.0': 0,
    **devicemap_TransformerUp('up.1.'),
    **devicemap_TransformerUp('up.2.'),
    **devicemap_TransformerUp('up.3.'),
    's_in': 0,
    's_out': 0,
}

save(device_map, 'unet')

In [3]:
def devicemap_layers(prefix):
    return {
        prefix + 'atten': 'disk',
        prefix + 'mlp': 0,
        prefix + 'norm1': 0,
        prefix + 'norm2': 0,
        prefix + 'scala1': 0,
        prefix + 'scala2': 0
    }


device_map = {'embed': 0, 'norm': 0}

for i in range(24):
    device_map = device_map | devicemap_layers('layers.%d.' % i)

save(device_map, 'encoder')

In [4]:
device_map = {
    'timestep_embedding': 0,
    'unet_s_in': 0,
    **devicemap_TransformerDown('unet_down.0.'),
    **devicemap_TransformerDown('unet_down.1.'),
    **devicemap_TransformerDown('unet_down.2.'),
    'unet_down.3': 0,
    **devicemap_Transformer2D('unet_mid.transformer.'),
    'unet_mid.resnet1': 0,
    'unet_mid.resnet2': 0,
    'embedding': 0,
    'controlnet_down': 0,
    'controlnet_mid': 0,
}

save(device_map, 'controlnet')